# Convex optimization: from simplest and intuitive methods to complicated and advanced tools

# Note

This project is based on my course paperwork 'Gradient descent algorithm and its modifications: from theory to practice and applications'. It is a practical survey made in Latex. The idea was to provide intuitive, mathematical and programming explanation of tools used for minimizing convex loss functions. Here I want to provide a summary of that work in Jupyter Notebook format

# 1 Introduction

### 1.1 Consider the simplest case


###### Suppose we have an independent vector and a dependent one

In [1]:
# choose arbitrary vectors
import numpy as np
x = np.array([1,3,5,7,9])
y = np.array(x**2 + 4 * x + 7)
print(x)
print(y)

[1 3 5 7 9]
[ 12  28  52  84 124]


I have written the function $y(x)$. However, suppose we do not know it explicitly. 
By taking a closer look we can see that the dependence is either linear or quadatric.
Cubic case is not very likely to be the one since $7^3=343>>84$; $9^3=729>>124$.
It still might be due to constants but we will not consider it.
We will now compare the performance of linear and quadratic cases by looking at the Euclidian norms

### 1.2 Linear case

###### In this classical case we approximate y as a linear combination of x:

### $\hat{y} = ax + b$, 

where $\hat{y}$ is predicted value of y; a and b are parameters on which it depends on

###### Now we can find these parameters using scipy optimization methods (probably the simplest way to do this)

In [2]:
from scipy.optimize import curve_fit
def func(x, a, b):
    return a * x + b
popt, pcov = curve_fit(func, x, y, p0=(1,1)) #we need to put initial guesses, let a = b = 1
popt # coefficients

array([ 14., -10.])

If we approximate function linearly, scipy tells us we get

### $\hat{y} = 14x - 10$

### 1.3 Quadratic case

This is indeed the right case in the problem since I have created it by myself. Here

### $\hat{y} = ax^2 + bx + c$. 

It is time to find those parameters

In [3]:
def func(x, a, b, c):
    return a * x ** 2 + b * x + c
popt, pcov = curve_fit(func, x, y, p0=(1,1,1)) 
popt 

array([1., 4., 7.])

If we put those coefficients to the function, we get that

### $\hat{y} = x^2 + 4x + 7$

We got the initial function and this proves the method works.

Nevertheless, it is still a good idea to compare the methods because usually the function is not known

### 1.4 Possible way of comparing methods - Euclidian norm

To do so we will define a cost function which we usually want to minimize

###### The approximation, resulting in lower loss, is better

### $cost = \sum_{i=1}^n (y - \hat{y})^2$

n - number of observations, here we have 5. Usually with deal with another versions of cost function, but here this one will work just fine

We see that it is exactly the Euclidian norm squared

So the function with lower Euclidian norm is better (we know that quadratic norm should be 0 since we just substract a function from itself but I want just to show that)

In [4]:
linear_norm = np.linalg.norm(y - np.array(14 * x - 10))
quadratic_norm = np.linalg.norm(y - np.array(x ** 2 + 4 * x + 7))
print(f'The value of linear norm is {linear_norm}, The value of quadratic norm is {quadratic_norm}')
if linear_norm <= quadratic_norm:
    print('As we see, linear approximation is better')
else:
    print('As we see, quadratic approximation is better')

The value of linear norm is 14.966629547095765, The value of quadratic norm is 0.0
As we see, quadratic approximation is better


# 2 Introduction to Gradient Descent methods

### 2.1 Swich to iterative methods

It is time to introduce the Gradient Descent algorithm (I will sometimes denote it as GD or BGD). To do so, we will define another cost functions for both linear and quadratic cases

### 2.2 Start with linear case

### $cost_{linear} = \frac{1}{2n}\sum_{i=1}^n (b + ax - y)^2$

For solving the problem, we will code the simplest possible Gradient Descent algorithm

In [5]:
def gradient_descent(x,y):
    b = a = 0
    iterations = 501
    learning_rate = 0.05
    n = len(x)
    for i in range(iterations):
        y_predicted = a * x + b
        cost = (1/n) * sum([val**2 for val in (y-y_predicted)])
        ad = -(1/n)*sum(x*(y-y_predicted))
        bd = -(1/n)*sum(y-y_predicted)
        a = a - learning_rate * ad
        b = b - learning_rate * bd
        if i%50 == 0:
            print(f' current value of a is {a}, current value of b is {b}, iteration {i}, cost is {cost}')
    return a, b
gradient_descent(x,y)

 current value of a is 20.6, current value of b is 3.0, iteration 0, cost is 5212.8
 current value of a is 13.014378868447867, current value of b is -3.541607776734829, iteration 50, cost is 55.15732745561401
 current value of a is 13.456857494803327, current value of b is -6.440998487785213, iteration 100, cost is 47.94525035744757
 current value of a is 13.700692535626, current value of b is -8.038754639652971, iteration 150, cost is 45.75513054438215
 current value of a is 13.835061779601366, current value of b is -8.919224015420816, iteration 200, cost is 45.090048248353725
 current value of a is 13.909108125301298, current value of b is -9.404420909051161, iteration 250, cost is 44.888080092159036
 current value of a is 13.94991256201093, current value of b is -9.671796506735337, iteration 300, cost is 44.82674762795081
 current value of a is 13.972398506988378, current value of b is -9.819138155405156, iteration 350, cost is 44.8081225573618
 current value of a is 13.984789750742

(13.995381043795494, -9.9697337771222)

###### Within 500 iterations we got to values which are very close to [14, -10], which are optimum values due to results we got before

We need to take into account that this algorithm is the simplest and does not perform well

we still converge and get wanted result but do it slowly and with some risk of skipping the optimum point

### 2.3 Quadratic case

### $cost_{quadratic} = \frac{1}{2n}\sum_{i=1}^n (ax^2 + bx + c - y)^2$

Here I want to write a more complex and sophisticated code to improve the algorithm.

###### So there are several additions & improvements:

###### 1) Now the function of the algorithm depends on more parameters:

(start - initial guesses, learn_rate - learning rate, n_iter - number of iterstions, tolerance - minimal changes by iteration (this allows us not to make iterstions that are not actually needed due to their uselessness)); now we can easily control some things about the algorithm without changing the code

###### 2) There was defined a new_function which returns the gradient of our function

After that we can implement it in the algorithm. As a result it is required to write less rows of code and it looks more generalised as it is always less tedious to change the gradient instead of half of the code

###### 3) Some other small changes mostly related to 1) and 2)

In [6]:
def gradient_descent(gradient, x, y, start, learn_rate, n_iter, tolerance=1e-06,
    dtype="float64"
):
    dtype_ = np.dtype(dtype)
    x, y = np.array(x, dtype=dtype_), np.array(y, dtype=dtype_)
    vector = np.array(start, dtype=dtype_)
    learn_rate = np.array(learn_rate, dtype=dtype_)
    n_iter = int(n_iter)
    tolerance = np.array(tolerance, dtype=dtype_)
    iteration_counter = 0
    iteration_5thousand = 0
    # Performing the gradient descent loop
    for _ in range(n_iter):
        # Recalculating the difference
        diff = -learn_rate * np.array(gradient(x, y, vector), dtype_)
        # Checking if the absolute difference is small enough
        if np.all(np.abs(diff) <= tolerance):
            break
        # Updating the values of the variables
        vector += diff
        if iteration_counter == 5000:
            iteration_5thousand += 5000
            iteration_counter = 0
            print(f'vector is {vector}, iteration {iteration_5thousand}')
        iteration_counter += 1
    return vector if vector.shape else vector.item()
# define a function which returns our gradient
def returned_gradient(x, y, b): 
    res = b[0] * x ** 2 + b[1] * x + b[2] - y
    return (res * x ** 2).mean(), (res * x).mean(), res.mean()
gradient_descent(returned_gradient, x, y, start=[1,1,1],
learn_rate=0.001011, n_iter=50001)

vector is [0.89966666 5.14609987 4.37036782], iteration 5000
vector is [0.93300072 4.76531152 5.24413714], iteration 10000
vector is [0.95526278 4.51101906 5.82756321], iteration 15000
vector is [0.97012776 4.34122115 6.21713247], iteration 20000
vector is [0.9800535  4.22784252 6.47725833], iteration 25000
vector is [0.98668119 4.15213657 6.65095135], iteration 30000
vector is [0.99110667 4.10158568 6.76693084], iteration 35000
vector is [0.99406169 4.06783148 6.84437346], iteration 40000
vector is [0.99603483 4.0452929  6.89608398], iteration 45000
vector is [0.99735235 4.03024329 6.93061248], iteration 50000


array([0.99735235, 4.03024329, 6.93061248])

###### We see that within 50 000 iterations for a given learning rate we end up with following vector of values (rounded to two decimal places):

[1, 4.03, 6.93] which is very close to the optimal one [1, 4, 7]. Of course if we ran more iterations we would reach ever closer point to the optimum but it will be too long

### 2.4 Interpretation of intermediate results 

So, the algorithm works but there is a problem with the learning rate. If we choose it too big, we diverge (for example, if we set it to be 0.01 in the latter example); if we set it to be too small, we fail to converge (or it will take too many iterations). There are many ways to deal with this problem and actually this is what I am going to talk about in the nearest future

### 2.5 Some more things before we move to the SGD algorithm

Linear case we took a look at has a property that we can find the optimal solution explicitly by using Pseudo-inverse matrices (linear regression problem). Let's have a look at the most popular example with house price prediction

I will create observations on my own in order not to read csv or excel files with pandas and not to send you additional files except this jupiter notebook

Here, as usually, we have $\hat{y}$ which is the price we are trying to predict. It depends on several parameters: square feet, number of rooms, age and city and the function is linearly dependent on each of those parameters

### $\hat{y} = \sum_{i=1}^n \theta_i x_i$, where $x_1 = 1$ since $\theta_1$ is the intercept

###### We will solve a problem using two approaches:

1) Sklearn built-in linear regression function

2) Pseudo-inverse matrices

This two things are actually the same but in first case we will do it automatically, and in the second we calculate the matrix multiplication by hands. And in both cases we deal with categorical variables. Matrices do not understand what do we mean by 'Moscow' or 'Odintsovo', they need numbers. Suppose we say that 1 - 'Moscow', 2 - 'Saint Petersburg', 3 - 'Odintsovo'. It will result in mistakes because Python will think that 'Moscow' < 'Odintsovo' or that 'Moscow' and 'Saint Petersburg' add up to 'Odintsovo'. To deal with the problem we will use special types of variables - dummy variables

In [7]:
# first approach
import pandas as pd
from sklearn.linear_model import LinearRegression
square_feet = np.array([35,45,60,75,96,43,58,76,37,100,41,150,29,75,43])
number_of_rooms = np.array([1,1,2,3,4,1,2,3,1,4,1,7,1,3,1])
age = np.array([1,3,4,8,15,2,6,8,1,15,16,1,2,15,2])
#I put numbers, because I do not want tohave problems with matrices, in DataFrame I will substitute them for cities
city = np.array([1,1,1,1,1,2,2,2,2,2,3,3,3,3,3]) # 1 - 'Moscow', 2 - 'Saint Petersburg', 3 - 'Odintsovo'
first_price = np.array([11,12,15,18,22,12,14.5,18,11,23,10.5,34,9,17,12])
final_price = []
for i in range(len(first_price)):
    if city[i] == 1:
        final_price.append(first_price[i] * 1.15)
    elif city[i] == 2:
        final_price.append(first_price[i] * 0.9)
    elif city[i] == 3:
        final_price.append(first_price[i] * 0.8)
final_price = np.array(final_price)
independent_features = np.zeros(shape = [4,15])
independent_features[0] = square_feet
independent_features[1] = number_of_rooms
independent_features[2] = age
independent_features[3] = city
independent_features = independent_features.T
df2 = pd.DataFrame(independent_features, columns = ['square feet', 'number of rooms', 'age', 'town'])
df2.loc[df2['town'] == 1, 'town'] = 'Moscow'
df2.loc[df2['town'] == 2, 'town'] = 'Saint Petersburg'
df2.loc[df2['town'] == 3, 'town'] = 'Odintsovo'
dummies = pd.get_dummies(df2.town)
merged = pd.concat([df2.drop(['town'], axis = 1), dummies], axis = 1)
# If we use One Hot Encoding, we always need to drop one dummy column
merged = merged.drop(['Odintsovo'], axis =1 )
print(merged)
model = LinearRegression()
X = merged
y = final_price
model.fit(X, y)
print(f'the intercept of the model is {round(model.intercept_,2)} and the coefficients are {np.round(model.coef_,2)}')

    square feet  number of rooms   age  Moscow  Saint Petersburg
0          35.0              1.0   1.0       1                 0
1          45.0              1.0   3.0       1                 0
2          60.0              2.0   4.0       1                 0
3          75.0              3.0   8.0       1                 0
4          96.0              4.0  15.0       1                 0
5          43.0              1.0   2.0       0                 1
6          58.0              2.0   6.0       0                 1
7          76.0              3.0   8.0       0                 1
8          37.0              1.0   1.0       0                 1
9         100.0              4.0  15.0       0                 1
10         41.0              1.0  16.0       0                 0
11        150.0              7.0   1.0       0                 0
12         29.0              1.0   2.0       0                 0
13         75.0              3.0  15.0       0                 0
14         43.0          

In [8]:
# second approach
merged1 = np.array(merged)
X = merged1
# remember that x_1 = 1
x0 = np.ones((15,1))
X = np.hstack((x0, X))
print(X)
# calculate Pseudo-inverse and multiply it by y
coefficient = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
coefficient
print(f'the intercept of the model is {round(coefficient[0],2)} and the coefficients are {np.round(coefficient[1:],2)}')

[[  1.  35.   1.   1.   1.   0.]
 [  1.  45.   1.   3.   1.   0.]
 [  1.  60.   2.   4.   1.   0.]
 [  1.  75.   3.   8.   1.   0.]
 [  1.  96.   4.  15.   1.   0.]
 [  1.  43.   1.   2.   0.   1.]
 [  1.  58.   2.   6.   0.   1.]
 [  1.  76.   3.   8.   0.   1.]
 [  1.  37.   1.   1.   0.   1.]
 [  1. 100.   4.  15.   0.   1.]
 [  1.  41.   1.  16.   0.   0.]
 [  1. 150.   7.   1.   0.   0.]
 [  1.  29.   1.   2.   0.   0.]
 [  1.  75.   3.  15.   0.   0.]
 [  1.  43.   1.   2.   0.   0.]]
the intercept of the model is 2.1 and the coefficients are [0.13 0.84 0.02 5.8  1.9 ]


As a result, 

### $\hat{y} = 2.1 + 0.13x_2 + 0.84x_3 + 0.02x_4 + 5.8x_5 + 1.9x_6$

where $x_2, x_3, x_4$ - integer variables standing for square_feet, rooms, age;

$x_5, x_6$ - binary variables representing the city ($x_5$ - 'Moscow', $x_6$ - 'Saint Petersburg'). The value is 1 if the city the house is at suits the name of the variable, 0 if it is in another city. $x_5 = x_6 = 0$ means it is 'Odintsovo'. The interpretation of the results will take long time. However, if we take a closer look at what we got, we see that it makes sense

# 3 Advanced iterative methods for solving convex optimization problems

### 3.1 When Stochastic gradient descent (SGD) is needed (switch from GD to SGD)

Suppose we have a dataset of n observations, $f_i(x)$ - loss function with respect to the observation of index i, x - vector of parameters. By summing those functions, we get a total loss function

### $f(x) = \frac{1}{n} \sum_ {i=1} ^ n f_i(x)$

Now we can compute the gradient:

### $\nabla f(x) = \frac{1}{n} \sum_ {i=1} ^ n \nabla f_i(x)$

What we do then is uptade the vector x at each iteration using this method:

### $x^{(t+1)} = x^{(t)} - \eta \nabla f(x)$

Now let's consider a general example. We have n observations, m parameters the function depends on and T iterations of the GD algorithm. In that case we will have to calculate n * m * T terms. Choice of T depends on us, but if n and m are huge, algorithm will take much time and will be slow. The comptutational cost depends for each variable linearly grows with n. There is a way to make it constant and make algorithm n times faster. This is where we speak about SGD. At each iteration we randomly choose just 1 observation and calculate derivatives without taking into consideration all other observations.

The new introduced algorithm will also converge. The main idea is that gradient of the particular observation $\nabla f_i(x)$ is an unbiased estimate of the gradient $\nabla f(x)$ because

### $\mathbb{E}_i \nabla f_i(x) = \frac{1}{n} \sum_ {i=1} ^ n \nabla f_i(x) = \nabla f(x)$

So, it is efficient to use it. The new algorithm will be:

### $x^{(t+1)} = x^{(t)} - \eta \nabla f_i(x^{(t)})$

And now we are ready to code it

In [9]:
def sgd(
    gradient, x, y, start, learn_rate=0.1, batch_size=1, n_iter=50,
    tolerance=1e-06, dtype="float64", random_state=None
):
    # Setting up the data type for NumPy arrays
    dtype_ = np.dtype(dtype)
    x, y = np.array(x, dtype=dtype_), np.array(y, dtype=dtype_)
    n_obs = x.shape[0] #number of elements in the vector in case of 1d and number of rows in case of 2d
    xy = np.c_[x.reshape(n_obs, -1), y.reshape(n_obs, 1)]
    # .reshape() to make sure that both x and y become two-dimensional arrays 
    # with n_obs rows and that y has exactly one column
    # numpy.c_[] conveniently concatenates the columns of x and y into a single array, xy. 
    #This is one way to make data suitable for random selection.
    
    # Initializing the random number generator
    seed = None if random_state is None else int(random_state)
    rng = np.random.default_rng(seed=seed) #since we do not use random state, different numbers all the time
    vector = np.array(start, dtype=dtype_)
    learn_rate = np.array(learn_rate, dtype=dtype_)
    batch_size = int(batch_size)
    n_iter = int(n_iter)
    tolerance = np.array(tolerance, dtype=dtype_)
    iteration_counter = 0
    iteration_5thousand = 0
    for _ in range(n_iter):
        rng.shuffle(xy) # a way to choose minibatches randomly
        for start in range(0, n_obs, batch_size):
            stop = start + batch_size
            x_batch, y_batch = xy[start:stop, :-1], xy[start:stop, -1:]
            grad = np.array(gradient(x_batch, y_batch, vector), dtype_)
            diff = -learn_rate * grad
            if np.all(np.abs(diff) <= tolerance):
                break
            vector += diff
        if iteration_counter == 5000:
            iteration_5thousand += 5000
            iteration_counter = 0
            print(f'vector is {vector}, iteration {iteration_5thousand}')
        iteration_counter += 1
    return vector if vector.shape else vector.item()
# Apply to the previous quadratic result
x = np.array([1,3,5,7,9])
y = np.array(x**2 + 4 * x + 7)
def returned_gradient(x, y, b): 
    res = b[0] * x ** 2 + b[1] * x + b[2] - y
    return (res * x ** 2).mean(), (res * x).mean(), res.mean()
sgd(returned_gradient, x, y, start=[1,1,1],
learn_rate=0.0005, n_iter=50001)

vector is [0.94601733 4.60191576 5.56858407], iteration 5000
vector is [0.97668389 4.21306684 6.49454394], iteration 10000
vector is [0.99199708 4.07629051 6.81764439], iteration 15000
vector is [0.99854278 4.02722549 6.93461682], iteration 20000
vector is [0.99922692 4.00976972 6.97614736], iteration 25000
vector is [0.99961731 4.00374372 6.99112666], iteration 30000
vector is [0.99989904 4.00149366 6.99635631], iteration 35000
vector is [0.99992526 4.00108509 6.99707505], iteration 40000
vector is [0.99993979 4.00095661 6.99725167], iteration 45000
vector is [0.99992229 4.00090532 6.99742005], iteration 50000


array([0.99992229, 4.00090532, 6.99742005])

### 3.2 Some interesting changes in terms of code (in comparison with the previuos algorithm):

###### 1) Function depends on presence or absence of random state
This state allows us to choose the same samples all the time but we do not now need it, therefore, do not use

###### 2) Batch_size is the number of observations we choose to perform an iteration
Actually, SGD is a particular case of mini batch gradient descent algorithm. By definition of it, we choose a subset of the set of observations to save time. So, SGD is just the most extreme case of this where the size of subset is just one random sample point.

###### 3) Everything is done randomly (in terms of choosing samples)
This minimizes risks of errors and inconveniences. There was introduced one way to do this by making a merged vector xy

Now we can compare the effectiveness of this algorithm with batch gradient descent. The results of the latter algorithm within the same number of iterations and the given learning rate which is now smaller are better than they were before since we got [1, 4, 7] which are indeed optimal values and previously we were just very close to them. Morevoer, SGD algorithm is 5 times faster (not exactly but very close to since we still need some comptutational cost to choose this random samples throughout iterations) since the number of observations is 5. That is why, it is definetely worth using. Note that depending on the situation, the SGD algorithm due to specificity of particular functions and randomness can require either less or bigger number of iterations to converge. In this particular example 50 000 iterations of the GD cost approximately as much as 250 000 iterations of SGD. That is why, if SGD will perform better within 250 000 iterations than SG within 50 000, we should use it. In examples I saw, SGD was always the best case.

### 3.3 Can we do even better?

Definitely, yes. Since GD and SGD are very sophisticated and well-studied algorithms, there are many possible modifications

### 3.4 Stochastic gradient descent with momentum

By introducing an additional term we will be able to remember what happened in the previous iterations and make algorithm faster. Now the formula is updated

### $x_{i+1} = x_i - \eta \nabla f_i (x_i) + \alpha \Delta x_i$

### $\Delta x_i = x_i - x_{i-1} = \alpha \Delta x_{i-1} - \eta \nabla f_i(x_{i-1})$

So, we move not exactly in the direction of gradient, we also pay attention to what happened to gradient in the past. The parameter $0 \leq \alpha \leq 1$ is called decay rate and it represents the importance of previous moves. Now it is time to implement this in the algorithm

In [10]:
def sgd_d(
    gradient, x, y, start, learn_rate=0.1, decay_rate=0.3, batch_size=1,
    n_iter=50, tolerance=1e-06, dtype="float64", random_state=None
):
    dtype_ = np.dtype(dtype)
    x, y = np.array(x, dtype=dtype_), np.array(y, dtype=dtype_)
    n_obs = x.shape[0]
    xy = np.c_[x.reshape(n_obs, -1), y.reshape(n_obs, 1)]
    seed = None if random_state is None else int(random_state)
    rng = np.random.default_rng(seed=seed)
    vector = np.array(start, dtype=dtype_)
    learn_rate = np.array(learn_rate, dtype=dtype_)
    # Setting up the decay rate
    decay_rate = np.array(decay_rate, dtype=dtype_)
    batch_size = int(batch_size)
    n_iter = int(n_iter)
    tolerance = np.array(tolerance, dtype=dtype_)
    # Setting the difference to zero for the first iteration
    diff = 0
    iteration_counter = 0
    iteration_5thousand = 0
    for _ in range(n_iter):
        rng.shuffle(xy)
        for start in range(0, n_obs, batch_size):
            stop = start + batch_size
            x_batch, y_batch = xy[start:stop, :-1], xy[start:stop, -1:]
            # Recalculating the difference
            grad = np.array(gradient(x_batch, y_batch, vector), dtype_)
            diff = decay_rate * diff - learn_rate * grad
            vector += diff
        if iteration_counter == 5000:
            iteration_5thousand += 5000
            iteration_counter = 0
            print(f'vector is {vector}, iteration {iteration_5thousand}')
        iteration_counter += 1
    return vector if vector.shape else vector.item()
# Apply to the previous quadratic result
x = np.array([1,3,5,7,9])
y = np.array(x**2 + 4 * x + 7)
def returned_gradient(x, y, b): 
    res = b[0] * x ** 2 + b[1] * x + b[2] - y
    return (res * x ** 2).mean(), (res * x).mean(), res.mean()
sgd_d(returned_gradient, x, y, start=[1,1,1],
learn_rate=0.0005, n_iter=50001)

vector is [1.08102775 5.16833988 5.35991633], iteration 5000
vector is [0.94333543 4.44138849 6.07255598], iteration 10000
vector is [1.10878676 4.21714837 6.6557175 ], iteration 15000
vector is [0.99481088 4.05938771 6.87905713], iteration 20000
vector is [0.99892059 4.02350591 6.92164191], iteration 25000
vector is [1.00080183 4.0092069  6.97882448], iteration 30000
vector is [0.99938713 4.01295522 6.9804388 ], iteration 35000
vector is [0.99985415 4.00286301 6.99439914], iteration 40000
vector is [1.00006926 3.99989567 6.99848159], iteration 45000
vector is [0.99994639 4.00064369 6.99880139], iteration 50000


array([0.99994639, 4.00064369, 6.99880139])

### 3.5 Important observation about convergence / divergence
Now I want to experiment. In SGD with no momentum we got very close to optimum (by this I mean when the last parameter exceeds 6.99 since it is the slowest out of those 3 to converge) within 30 000 iteration. Now let's set decay_rate to be 0.2; the same result is got within 27 000 iteration. Now set decay_rate to be 0.5. As a result, our values diverge. Within $\alpha = 0.3$ we converged already on 19 000 iteration. As we see, the method works. However, it is vital not to set the parameter being too big. It will result in the divergence of the estimated parameters (the same problem as with learning rate). Note that we can get similar but not exactly same results due to randomness and the absence of random state which can ensure us having same results. However, I still do not see any point in it.

### 3.6 What else can we do?

The main problem of GD in its various terms is the learning rate. If we set it to be too big - we diverge; if we set it to be too small - we either will be stuck near some point and will not converge or we will converge but not in the optimal way (as the same things could have been done much faster). What I did before is just experimenting with different values, setting them to be constant throughout the entire fulfillment of the algorithm and after that choosing those I have considered to be suitable for both avoiding the divergence and getting relatively fast convergence. However, this is not the best way to do it.

### 3.7 The better approach is rescaling the learning rate at each iteration

###### There are two heuristics:

1) when cost function value decreases after an iteration, the step could have been larger. That is why, we try to increase it

2) when cost function value increases after a step, the step-size was too large. That is why we need to undo the step with the descreased learning rate.

### 3.8 How do we change the step-size?

There are some basic strategies to adjust learning rate over time (piecewise constant, exponential decay, polynomial decay) as well as more complex ones. The idea of complex methods is that now learning rate becomes a time dependent decreasing function. The further we go, the closere we get to the optimum and the more careful we need to be since in case we skip the optimum our loss function will start to increase and go to infinity

### 3.9 RMSprop

We will not talk about AdaGrad and move straight to its imrpoved version. Starting from now, I will not explain the way algorithms can be coded - in my article this section is an Appendix one 

###### The basic formula remains the same

### $x^{(t+1)} = x^{(t)} - \eta \nabla f(x^{(t)})$.

###### However, now the learning rate is not a constant, it is a function:

### $\eta_t = \frac{\eta_0}{\sqrt{w_{avg} + \epsilon}}$,

where 

### $w_{avg_t} = \zeta w_{avg_{t-1}} + (1-\zeta) (\nabla f(x^{(t)}))^2$.

$w_{avg_t}$ stands for the weighted average at time t. The parameter $\zeta$ represents for how much at a particular step we need to take into consideration the previous step and for how much do we need to take into consideration the newly calculated gradient

### 3.10 Adam as a combination of RMSprop and Momentum

Probably one of the best modifications of the SGD algorithm and very widely used technique in Deep Learning for training neural networks is the Adam method. Despite it is usually interpreted as a rather complicated and advanced one, it will not be challenging for us since we are already familiar with everything needed to study it. Remember two important formulas: 

### 1) $x^{(t+1)} := x^{(t)} - \eta * v_{dx^{(t+1)}}$,

where 

### $v_{dx^{(t+1)}} = \gamma * v_{t} + (1- \gamma) * \nabla f_i(x^{(t)})$ 

### 2) $x^{(t+1)} := x^{(t)} - \frac{\eta * \nabla f_i(x^{(t+1)})}{\sqrt{w_{avg_t} + \epsilon}}$, 

where 

### $w_{avg_t} = \zeta w_{avg_{t-1}} + (1-\zeta) (\nabla f_i(x^{(t)}))^2$ 

The first one is the representation of the momentum method, while the second one stands for the RMSprop optimizer. Now the only thing left is coming up with a formula combining this two elements. Here it is:

### $x^{(t+1)} := x^{(t)} - \eta * \frac{v_{dx^{(t+1)}}}{\sqrt{w_{avg_{t+1}}+ \epsilon}}.$


###### This is the Adam method. The inventors of the algorithm in their book recommend using $\gamma = 0.9$ and $\zeta = 0.999$. 

# 4 Conclusion


Well, we started with the introduction to types of problems which include convex optimization. After that, we gave a brief talk about how these problems can be solved and moved to iterative methods. Then, it turned out that gradient descent algorithm is one of the best and universal solutions for the convex optimization problems. Despite all its advantages, the simplest form had many drawbacks we discussed. That is why we moved to the most important part of the research - mathematical and intuitive explanation and programming of several methods which help to overcome mentioned issues. Finally, we understood how to combine all of them (SGD, Momentum, Adaptive Learning Rate) in one tool - Adam, which is found one of the most widely used and powerful nowadays. 

# 5 What else I have done while studying machine learning?

The GD and SGD algorithms in their variations were the key ML topics for a long time since they were related to the university project and those topics we discussed and studied with my scientific director. However, I am interested in the other parts of ML as well as DL too (I believe that in the future neural networks will play an even more important role in our lives, so it is necessary to understand the way they are used), that is why I do study maths and codes of other algorithms. Despite that I am not doing it for a long time, by today I have already had experience with digits recognition with the help of logistic regression, iris classification using SVM (linear kernel) and clusterization of Kinopoisk TV-series using K-means, Hierarhical and Louvain algorithms (it was done as a home-work mini-project). What is more, I am familiar with basic concepts and terminology, such as overfitting, regularization, accuracy, ROC and AUC and so on.

# 6 What are the plans for the future?

1) Continue working on the project. I want to move to the applied part, since SGD is commonly used in Deep Learning, which I am interested in

2) Continue studying advanced Machine Learning algorithms (both mathematics of them and coding)

3) Use GD and SGD while implementing other algorithms (since most of the time the ultimate problem is to minimize a certain convex function and here what I am currently studying is the key helper)

 # 7 References
 1) https://d2l.ai/chapter_optimization/sgd.html (unbiased estimate and information about the update of the learning rate)
 
 2) https://realpython.com/gradient-descent-algorithm-python/ (how to code GD and SGD & some basic ideas about it)
 
 3) codebasics (probably one of the best Youtube channels to start learning ML in Python)
 
 4) StatQuest (Youtube channel where one can find intuitive explanations of ML algorithms)
 
 5) stanfordonline, CS229 (A course which gives comprehensive view over ML algorithms)
 
 6) Mathematics for Machine Learning (a book which allows to master all the necessary prerequisites for studying machine learning)
 
 7) DeepLearningAI courses (the explanation of RMSprop and Adam methods)

# Thank you for the attention!